In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
tf.__version__ 

'2.8.0'

# Part 1 - Data Preprocessing

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [5]:
training_set = train_datagen.flow_from_directory('./ASL_DATASET/dataSet/trainingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 12845 images belonging to 27 classes.


In [6]:
test_set = test_datagen.flow_from_directory('./ASL_DATASET/dataSet/testingData',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 4268 images belonging to 27 classes.


# Part 2 - Building the CNN

## Initializing the CNN

In [7]:
classifier = tf.keras.models.Sequential()

2023-02-18 09:33:34.650236: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Step 1 - Convolution

In [8]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#### Step 2 - Pooling

In [9]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Adding a second convolutional layer

In [10]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [11]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [12]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

## Part 3 - Training the CNN

### Compiling the CNN

In [13]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 128)               4

In [17]:
classifier.fit(training_set,
                  epochs = 15,
                  validation_data = test_set)

Epoch 1/15
1285/1285 [==============================] - 196s 152ms/step - loss: 0.3334 - accuracy: 0.8920 - val_loss: 0.0188 - val_accuracy: 0.9958
Epoch 2/15
1285/1285 [==============================] - 188s 146ms/step - loss: 0.3089 - accuracy: 0.8989 - val_loss: 0.0171 - val_accuracy: 0.9963
Epoch 3/15
1285/1285 [==============================] - 190s 147ms/step - loss: 0.2609 - accuracy: 0.9167 - val_loss: 0.0226 - val_accuracy: 0.9941
Epoch 4/15
1285/1285 [==============================] - 188s 146ms/step - loss: 0.2389 - accuracy: 0.9237 - val_loss: 0.0125 - val_accuracy: 0.9974
Epoch 5/15
1285/1285 [==============================] - 187s 145ms/step - loss: 0.2262 - accuracy: 0.9281 - val_loss: 0.0096 - val_accuracy: 0.9965
Epoch 6/15
1285/1285 [==============================] - 187s 146ms/step - loss: 0.2239 - accuracy: 0.9304 - val_loss: 0.0086 - val_accuracy: 0.9979
Epoch 7/15
1285/1285 [==============================] - 192s 149ms/step - loss: 0.1927 - accuracy: 0.9415 - val_

# Saving the Model

In [19]:
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new.h5')
print('Weights saved')

Model Saved
Weights saved


In [41]:
pwd

'/home/u184497/ASL'

SyntaxError: invalid syntax (<string>, line 1)

In [3]:
import streamlit as st
from keras.models import load_model
import cv2
import numpy as np
import os
import mediapipe as mp
mphands = mp.solutions.hands
hand = mphands.Hands(max_num_hands=2, min_detection_confidence=0.75)
mpdraw = mp.solutions.drawing_utils
st.title("Sign Language Classification")
st.header("Gesture Model")
all_classes = os.listdir("./ASL")


@st.cache(hash_funcs={'tensorflow.python.keras.utils.object_identity.ObjectIdentityDictionary': id}, allow_output_mutation=True)
def model_upload():
    model = load_model("./model_new.h5")
    print("Loading")
    return model


def predict(model, image):
    img = np.array(image)
    img = cv2.resize(img, (100, 100))
    img = img / 255
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    pred = np.argmax(prediction, axis=-1)
    return pred, prediction[0][pred[0]]


run = st.checkbox('Run')
FRAME_WINDOW = st.image([])
camera = cv2.VideoCapture(0)
if run is False:
    camera.release()

while run:
    x_points = []
    y_points = []
    _, frame = camera.read()
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hand.process(frame_rgb)
    if results.multi_hand_landmarks:
        for landmarks in results.multi_hand_landmarks:
            for id, ld in enumerate(landmarks.landmark):
                h, w, channels = frame.shape
                x_points.append(int(ld.x * w))
                y_points.append(int(ld.y * h))

            a1 = (int(max(y_points) + 30), int(min(y_points) - 30))
            a2 = (int(max(x_points) + 30), int(min(x_points) - 30))
        cv2.rectangle(frame, (a2[1], a1[1]), (a2[0], a1[0]), (0, 255, 0), 3)
        if len(x_points) == 21 or len(x_points) == 42:
            target = frame[a1[1]:a1[0], a2[1]:a2[0]]

            if len(target) > 0:
                m = model_upload()
                p, num = predict(m, frame)
                cv2.putText(frame, str(all_classes[p[0]]) + " " + str(100*num), (80, 80), cv2.FONT_ITALIC, 2, (255, 100, 100), 2)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    FRAME_WINDOW.image(frame_rgb)


else:
    st.write('Stopped')
    camera.release()

<class 'ModuleNotFoundError'>: No module named 'keras'